# **Importación de Librerías**

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision import datasets
from torchvision import transforms
from torchvision.io import read_image
from torchvision.transforms import ToTensor, Lambda, Compose

from tqdm.notebook import tqdm

# **Carga de Datos**

In [2]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])

train_set_orig = datasets.FashionMNIST('MNIST_data/', download=True, train=True , transform=transform)
valid_set_orig = datasets.FashionMNIST('MNIST_data/', download=True, train=False, transform=transform)

# **CustomDataset**

In [3]:
class CustomDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        input = output = self.dataset[idx][0]
        return input, output

# **Funciones**

## Una época de entrenamiento

In [4]:
## TODO: Probablemente tenga que agregar para que compute la accuracy
def train_step(model, dataloader, loss_fn, optimizer):
    """
    Trains the model for ONE epoch = goes over the dataset one time using the batches.
    """
    num_batches = len(dataloader)
    total_loss = 0
    
    model_device = next(model.parameters()).device
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(model_device), y.to(model_device)

        y_pred = model(X)
        loss = loss_fn(y_pred, y)

        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            print(f"Batch {batch} of {num_batches}. Loss in batch: {loss.item():.4f}")
    
    avg_loss = total_loss / num_batches

    return avg_loss

## Validación post época

In [5]:
## TODO: Probablemente tenga que agregar para que compute la accuracy. Capaz con
## un if en la función de pérdida

def validate_step(model, dataloader, loss_fn):
    num_batches = len(dataloader)
    total_loss = 0

    model_device = next(model.parameters()).device
    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(model_device), y.to(model_device)

            y_pred = model(X)
            total_loss += loss_fn(y_pred, y).item()

    avg_loss = total_loss / num_batches

    return avg_loss

## Loop de entrenamiento - validación

In [6]:
def train_validate_loop(
    model, train_dataloader, valid_dataloader, loss_fn, optimizer, epochs
):
    train_avg_losses_training, train_avg_losses, valid_avg_losses = [], [], []

    for epoch in tqdm(range(1, epochs+1)):
        tqdm.write(f"Epoch {epoch}")
        train_avg_loss_training = train_step(model, train_dataloader, loss_fn, optimizer)

        train_avg_loss = validate_step(model, train_dataloader, loss_fn)
        valid_avg_loss = validate_step(model, valid_dataloader, loss_fn)

        tqdm.write(f"Train avg loss: {train_avg_loss:.6f}")
        tqdm.write(f"Valid avg loss: {valid_avg_loss:.6f}")
        tqdm.write("----------------------------------------------------------------")

        train_avg_losses_training.append(train_avg_loss_training)
        train_avg_losses.append(train_avg_loss)
        valid_avg_losses.append(valid_avg_loss)

    print(f"Training finished! Trained for {epoch} epochs.")
    print(
        f"Final results from epoch {epoch}:\n"
        f"  - Train avg loss: {train_avg_loss:.6f}\n"
        f"  - Valid avg loss: {valid_avg_loss:.6f}"
    )
    return model, train_avg_losses_training, train_avg_losses, valid_avg_losses

## Gráfico de errores

In [7]:
def losses_plot(
    train_avg_losses_training, train_avg_losses, valid_avg_losses
):
    epochs = len(train_avg_losses)
    epochs_range = range(epochs)
    
    plt.figure(figsize=(8, 6))
    plt.plot(epochs_range, train_avg_losses_training, label="Durante la época", linestyle="--", color="blue")
    plt.plot(epochs_range, train_avg_losses, label="Entrenamiento", linestyle="-", color="green")
    plt.plot(epochs_range, valid_avg_losses, label="Validación", linestyle=":", color="red")
    
    plt.title("Pérdidas durante el entrenamiento")
    plt.xlabel("Épocas")
    plt.ylabel("Pérdida promedio por lote")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.show()

## Imagen original y reconstruida

In [8]:
def plot_orig_predicted(model, train_set, num_samples=3):
    model_device = next(model.parameters()).device
    model.eval()

    num_samples = 2
    fig, axes = plt.subplots(nrows=num_samples, ncols=2, figsize=(8, 2*num_samples))

    for i in range(num_samples):
        sample_idx = torch.randint(len(train_set), size=(1,)).item()
        input = train_set[sample_idx][0].unsqueeze(1).to(model_device)
        
        with torch.no_grad():
            output = model(input).squeeze(1).cpu().numpy()

        # Plot original image
        ax_orig = axes[i, 0]
        ax_orig.imshow(input.squeeze().cpu().numpy(), cmap='gray')
        ax_orig.axis('off')
        ax_orig.set_title(f"Original {sample_idx}")
        
        # Plot predicted image
        ax_pred = axes[i, 1]
        ax_pred.imshow(output.squeeze(), cmap='gray')
        ax_pred.axis('off')
        ax_pred.set_title(f"Reconstructed {sample_idx}")

## Shape post-convolución

Shape:
* Input: $(C_{in}, H_{in}, W_{in})$
* Output: $(C_{out}, H_{out}, W_{out})$,

where:

\begin{align*}
H_{out} = \left\lfloor \frac{H_{in} + 2 \times \text{padding}[0] - \text{dilation}[0] \times (\text{kernel\_size}[0] - 1) - 1}{\text{stride}[0]} + 1 \right\rfloor
\\
W_{out} = \left\lfloor \frac{W_{in} + 2 \times \text{padding}[1] - \text{dilation}[1] \times (\text{kernel\_size}[1] - 1) - 1}{\text{stride}[1]} + 1 \right\rfloor
\end{align*}

In [9]:
def post_conv_shape(
    h_in,
    w_in,
    out_channels,
    kernel_size,
    stride=1,
    padding=0,
    dilation=1
):
    h_out = int(((h_in + 2 * padding - dilation * (kernel_size - 1) - 1) / stride) + 1)
    w_out = int(((w_in + 2 * padding - dilation * (kernel_size - 1) - 1) / stride) + 1)
    return out_channels, h_out, w_out

## Shape post-convolución transpuesta

Shape:
* Input: $(C_{in}, H_{in}, W_{in})$
* Output: $(C_{out}, H_{out}, W_{out})$,

where:

\begin{align*}
H_{out} = (H_{in} - 1) \times \text{stride}[0] - 2 \times \text{padding}[0] + \text{dilation}[0] \times (\text{kernel\_size}[0] - 1) + \text{output\_padding}[0] + 1
\\
W_{out} = (W_{in} - 1) \times \text{stride}[1] - 2 \times \text{padding}[1] + \text{dilation}[1] \times (\text{kernel\_size}[1] - 1) + \text{output\_padding}[1] + 1
\end{align*}

In [10]:
def post_transconv_shape(
    h_in,
    w_in,
    out_channels,
    kernel_size,
    stride=1,
    padding=0,
    output_padding=0,
    dilation=1
):
    h_out = (h_in - 1) * stride - 2 * padding + dilation * (kernel_size - 1) + output_padding + 1
    w_out = (w_in - 1) * stride - 2 * padding + dilation * (kernel_size - 1) + output_padding + 1
    return out_channels, h_out, w_out

## Shape post-max pooling

Shape:
* Input: $(C_{in}, H_{in}, W_{in})$
* Output: $(C_{out}, H_{out}, W_{out})$,

where:

\begin{align*}
H_{out} = \left\lfloor \frac{H_{in} + 2 \times \text{padding}[0] - \text{dilation}[0] \times (\text{kernel\_size}[0] - 1) - 1}{\text{stride}[0]} + 1 \right\rfloor
\\
W_{out} = \left\lfloor \frac{W_{in} + 2 \times \text{padding}[1] - \text{dilation}[1] \times (\text{kernel\_size}[1] - 1) - 1}{\text{stride}[1]} + 1 \right\rfloor
\end{align*}

In [11]:
def post_maxpool_shape(
    h_in,
    w_in,
    out_channels,
    kernel_size,
    stride=None,
    padding=0,
    dilation=1
):
    if stride is None:
        stride = kernel_size
    h_out = int(((h_in + 2 * padding - dilation * (kernel_size - 1) - 1) / stride) + 1)
    w_out = int(((w_in + 2 * padding - dilation * (kernel_size - 1) - 1) / stride) + 1)
    return out_channels, h_out, w_out

# **1) Red Neuronal Autoencoder Convolucional de varias capas**
Defina y cree una red neuronal autoenconder convolucional. El autoencoder debe poseer dos módulos, un
encoder y un decoder. 
* El encoder tienen que tener al menos dos capas convolucionales 2D y una lineal.
* El decoder tiene que realizar una transformación aproximadamente inversa, por ejemplo, utilizando
primero una capa lineal y luego dos convolucionales traspuestas.
Recuerde incluir dropout, si es que lo considera necesario, y elegir adecuadamente el tipo de unidades de
activación de la capa de salida. No utilice dropout en la capa de salida. Por qué?

In [35]:
kernels       = [3 , 3 ]
paddings      = [0 , 0 ]
strides       = [1 , 1 ]
out_channels  = [16, 32]

ch_in, h_in, w_in = 1, 28, 28

for i, (kernel_size, padding, stride, out_channel) in enumerate(zip(kernels, paddings, strides, out_channels)):
    ch_out, h_out, w_out = post_conv_shape(h_in, w_in, out_channel, kernel_size, stride, padding)
    print(f"Conv{i+1}:    ({ch_in}, {h_in}, {w_in}) -> ({ch_out}, {h_out}, {w_out})")
    ch_in, h_in, w_in = ch_out, h_out, w_out

    ch_out, h_out, w_out = post_maxpool_shape(h_in, w_in, out_channel, kernel_size=2)
    print(f"MaxPool{i+1}: ({ch_in}, {h_in}, {w_in}) -> ({ch_out}, {h_out}, {w_out})\n")
    ch_in, h_in, w_in = ch_out, h_out, w_out

print(f"Final encoder shape: ({ch_out}, {h_out}, {w_out})")

post_transconv_shape(h_in, w_in, 16, kernel_size=3, stride=2, output_padding=2)

post_transconv_shape(13, 13, 1, kernel_size=3, stride=2, output_padding=1)

Conv1:    (1, 28, 28) -> (16, 26, 26)
MaxPool1: (16, 26, 26) -> (16, 13, 13)

Conv2:    (16, 13, 13) -> (32, 11, 11)
MaxPool2: (32, 11, 11) -> (32, 5, 5)

Final encoder shape: (32, 5, 5)


(1, 28, 28)

In [36]:
class ConvAutoencoder(nn.Module):
    def __init__(self, p):
        super().__init__()
        self.encoder = nn.Sequential(
            # First Convolution
            # (1,28,28) -> (16,26,26)
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3),
            nn.ReLU(),
            nn.Dropout(p=p),
            # (16,26,26) -> (16,13,13)
            nn.MaxPool2d(kernel_size=2),
            # Second Convolution
            # (16,13,13) -> (32,11,11)
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3),
            nn.ReLU(),
            nn.Dropout(p=p),
            # (32,11,11) -> (32,5,5)
            nn.MaxPool2d(kernel_size=2)
        )
        self.linear = nn.Sequential(
            # (32,5,5) -> 32x5x5
            nn.Flatten(),
            # 32x5x5 -> 32x5x5
            nn.Linear(in_features=32*5*5, out_features=32*5*5),
            nn.ReLU(),
            nn.Dropout(p=p)
        )
        self.decoder = nn.Sequential(
            # En principio, vamos a pasar de: tamaño post maxpooling a tamaño
            # pre convolución
            nn.Unflatten(dim=1, unflattened_size=(32,5,5)),
            # First Deconvolution
            # (35,5,5) -> (16,13,13)
            nn.ConvTranspose2d(in_channels=32, out_channels=16, kernel_size=3, stride=2, output_padding=2),
            # Secon Deconvolution
            # (16,13,13) -> (1,28,28)
            nn.ConvTranspose2d(in_channels=16, out_channels=1, kernel_size=3, stride=2, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.linear(x)
        x = self.decoder(x)
        return x

# **2) Entrenando el autoencoder**

**2.1) Implemente en una función train_loop un loop que itere en modo entrenamiento sobre los batchs
o lotes de una época de entrenamiento.**

Hecho en función `train_step`.

**2.2) Implemente en una función eval_loop un loop que itere en modo evaluación sobre los batchs de
una  ́epoca de entrenamiento.**

Hecho en función `validate_step`.

**2.3) Inicialize los DataLoaders sobre el conjunto de entranmiento y el conjunto de validación, usando
batchs de 100 ejemplos. AYUDA: Crear nuevas clases derivadas de la clase Dataset que sirvan para
entrenar autoencoders, i.e. en donde tanto el input como el output sean la misma imagen.**

Hecho en clase `CustomDataset`.

2.4) Cree una función de pérdida usando el Error Cuadrático Medio (ECM).

2.5) Cree un optimizador con un learning rate igual a 10−3. Pruebe con ADAM.

2.6) Cree una instancia del modelo autoencoder.

2.7) Especifique en que dispositivo (device) va a trabajar. Lo har ́a en una CPU, en una GPU o una
TPU?

**2.8) Implemente un loop que itere sobre épocas de entrenamiento. Este loop debe guardar en listas
correspondientes, y en función de las épocas, los promedios del ECM sobre los conjuntos de entrenamiento y validación, inclueyendo los valores incorrectamente estimados del ECM sobre el conjunto de
entrenamiento. IMPORTANTE: No olvide copiar los batchs al dispositivo de trabajo.**

Hecho en función `train_validate_loop`.

2.9) Entrene y valide el modelo.

2.10) Grafique los distintos valores del ECM en función de las épocas de entrenamiento. Cual es el
número  ́optimo de  ́epocas de entrenamiento? Discuta y comente.

2.11) Grafique, comparativamente, algunas de las imagenes a predecir vs las imagenes predichas por el
modelo entrenado.

2.12) Repita utilizando variaciones de los hiperparámetros del autencoder. Por ejemplo, utilizando capas lineales de otros tamaños, valores distintos de dropout, otros optimizadores, distintos learning rates,
distintas transformaciones convolucionales y convolucionales traspuestas, etc. Que valores de estos hiperparámetros considera los más convenientes? Por qué?

# **3) Definiendo y entrenando un clasificador convolucional reutilizando el encoder**

3.1) Defina y cree un clasificador convolucional, agregando una capa clasificadora al encoder del
autoencoder previamente entrenado.

3.2) Reimplemente las funciones de entrenamiento, teniendo en cuenta que ahora debe incluir el c ́alculo
de precisiones.

3.3) Cree una función de pérdida usando la Cross Entropy Loss (CEL).

3.4) Cree una instancia del modelo clasificador.

3.5) Entrene y valide el modelo.

3.6) Grafique los distintos valores de la CEL y la precisi ́on calculados, en funci ́on de las  ́epocas de
entrenamiento.

3.7) Utilice el conjunto de validaci ́on para calcular una Matriz de confusi ́on. Graf ́ıquela y comente los
resultados.

# **4) Prentrenamiento**
Modifique el optimizador para que solo reentrene los par ́ametros de la capa clasificadora, dejando los
par ́ametros de la capa codificadora tal como vienen entrenada del el autoencoder convolucional. Repita
los experimentos de la parte 3. Qué observa? Comente

Ayuda: Se recomienda guardar en archivos los pesos de las distintas capas de las redes entrenadas para
que puedan ser reutilizadas.